# Setup

In [1]:
import pandas as pd
import altair as vg
import seaborn as sns
import numpy as np
import math as m

# Data Prep

In [2]:
filepath = '../data/Dados_v3.xlsx'
df = pd.read_excel(filepath)

In [3]:
df.head()

,Sigla,Nome,Latitude,Longitude,Município,Modal,Rio,Latitude_2,Longitude_2,OBS,Considerar,Carga_2023,Carga_2024,GN,GNL,Conteineres/Ano
0,AM-000,MANAUS,-3.138087,-60.027484,Manaus,NaN,Negro,-3.138087,-60.027484,Porto de Manaus,0,0,0,NaN,NaN,NaN
1,AM-083,SACAMBU,-3.272442,-60.952701,Manacapuru,Hidroviário,Solimões,-3.274633,-60.933956,"Lago Cabaliana, UTE Sacambu; este nó será agre...",0,1487,1511,4.558757e+05,759.792829,15.195857
2,AM-028,CAMPINAS,-3.278834,-61.099313,Manacapuru,Hidroviário,Solimões,-3.278834,-61.099313,Lago Cabaliana + Rio Paraná do Aramã; este nó ...,1,1085,1099,3.315734e+05,552.622316,11.052446
3,AM-023,CAAPIRANGA,-3.324601,-61.212903,Manacapuru,Gasoduto,Solimões,-3.278834,-61.099313,Verificar acesso ao município; supondo entrega...,0,9984,10253,3.093378e+06,5155.629306,103.112586
4,AM-004,ANAMÃ,-3.572283,-61.407035,Anamã,Gasoduto,Solimões,-3.572283,-61.407035,NaN,0,12675,13347,4.026852e+06,6711.419520,134.228390


In [4]:
df.dtypes

Sigla               object
Nome                object
Latitude           float64
Longitude          float64
Município           object
Modal               object
Rio                 object
Latitude_2         float64
Longitude_2        float64
OBS                 object
Considerar           int64
Carga_2023           int64
Carga_2024           int64
GN                 float64
GNL                float64
Conteineres/Ano    float64
dtype: object

# EDA

In [5]:
df.describe()

,Latitude,Longitude,Latitude_2,Longitude_2,Considerar,Carga_2023,Carga_2024,GN,GNL,Conteineres/Ano
count,86.000000,86.000000,86.000000,86.000000,96.000000,96.000000,96.000000,8.200000e+01,82.000000,82.000000
mean,-3.773587,-63.488361,-3.778263,-63.564175,0.625000,20103.364583,20673.541667,6.879314e+06,11465.524114,229.310482
std,2.076320,4.406264,2.087978,4.343704,0.486664,31821.860685,32843.231021,1.043984e+07,17399.725018,347.994500
min,-8.747551,-72.581236,-8.747551,-72.581236,0.000000,0.000000,0.000000,6.516820e+04,108.613667,2.172273
25%,-4.721653,-67.061608,-4.719700,-67.061608,0.000000,1568.250000,1626.750000,7.980842e+05,1330.140293,26.602806
50%,-3.354899,-62.973198,-3.350754,-63.020109,1.000000,10170.000000,10398.000000,3.806758e+06,6344.596973,126.891939
75%,-2.746235,-59.750366,-2.707575,-60.004683,1.000000,21500.500000,22015.500000,7.134032e+06,11890.053804,237.801076
max,1.190652,-56.668167,1.190652,-56.668167,1.000000,210942.000000,220434.000000,6.650596e+07,110843.264444,2216.865289


## Mapa de demanda

In [6]:
# Topo map
filepath = 'https://raw.githubusercontent.com/tbrugz/geodata-br/master/geojson/geojs-13-mun.json'

topo = vg.Chart(filepath).mark_geoshape(
    stroke="#dd",
    strokeWidth=0.3,
    color="black",
).project(
    type='mercator'
)

# Demanda
selection = vg.selection_multi(fields=['Rio'], bind='legend')
points = vg.Chart(df, title="Demanda em contêinereres por ano").mark_point().encode(
    vg.Longitude('Longitude:Q',
                ),
    vg.Latitude('Latitude:Q',
               ),
    color= vg.condition(selection,
                        vg.Color('Rio:N', scale=vg.Scale(scheme='category10')),
                        vg.value('lightgray'),
                       ),
    size=vg.Size('Conteineres/Ano:Q', 
                 scale=None,
                 legend=vg.Legend(title='Conteineres por Ano')),
    tooltip=['Sigla', 'Nome', 'Latitude', 'Longitude', 'Município', 'Modal', 'Rio',
             'Latitude_2', 'Longitude_2', 'OBS', 'Considerar', 'Carga_2023',
             'Carga_2024', 'GN', 'GNL', 'Conteineres/Ano']
)

# Mapa + Demanda + Configs
vg.layer(
    topo,
    points,
).properties(
    width=600,
    height=600,
).configure_title(
    fontSize=24,
).configure_axis(
    grid=False,
    labelFontSize=14,
    titleFontSize=20,
).configure_legend(
    titleFontSize=16,
    labelFontSize=14, 
).add_selection(
    selection
)

alt.LayerChart(...)

## Demanda (permite zoom e seleção na legenda)

In [7]:
selection = vg.selection_multi(fields=['Rio'], bind='legend')

chart = vg.Chart(df, title="Demanda em contêinereres por ano").mark_point().encode(
    vg.X('Longitude:Q',
          scale=vg.Scale(zero=False)
        ),
    vg.Y('Latitude:Q',
         scale=vg.Scale(zero=False)
        ),
    color= vg.condition(selection,
                        vg.Color('Rio:N', scale=vg.Scale(scheme='category10')),
                        vg.value('lightgray'),
                       ),
    size=vg.Size('Conteineres/Ano:Q', 
                 scale=None,
                 legend=vg.Legend(title='Conteineres por Ano')),
    tooltip=['Sigla', 'Nome', 'Latitude', 'Longitude', 'Município', 'Modal', 'Rio',
             'Latitude_2', 'Longitude_2', 'OBS', 'Considerar', 'Carga_2023',
             'Carga_2024', 'GN', 'GNL', 'Conteineres/Ano']
).properties(
    width=600,
    height=600,
).configure_title(
    fontSize=24,
).configure_axis(
    grid=False,
    labelFontSize=14,
    titleFontSize=20,
).configure_legend(
    titleFontSize=16,
    labelFontSize=14, 
).add_selection(
    selection
).interactive()

chart

alt.Chart(...)

## Pareto

In [8]:
df_pareto = df.copy()
df_pareto = df_pareto[df_pareto['Conteineres/Ano'].notna()]
df_pareto = df_pareto.sort_values(by=['Conteineres/Ano'], ascending=False)

df_pareto["count cumsum"] = df_pareto['Conteineres/Ano'].cumsum()
df_pareto["cumpercentage"] = df_pareto["count cumsum"]/(df_pareto['Conteineres/Ano'].sum())

sort_order = df_pareto["Nome"].tolist()

selection = vg.selection_multi(fields=['Rio'], bind='legend')

# Create Base
base = vg.Chart(df_pareto, title="Pareto da Demanda").encode(
    vg.X("Nome:O",
         sort=sort_order),
).properties(
    width=800,
    height=600,
)
# Bars chart
bars = base.mark_bar().encode(
    vg.Y("Conteineres/Ano:Q"),
    color= vg.condition(selection,
                        vg.Color('Rio:N', scale=vg.Scale(scheme='category10')),
                        vg.value('lightgray'),
                       ),
).properties(
    width=800,
    height=600,
)
# Line chart
line = base.mark_line(strokeWidth=1.5, color="#cb4154" ).encode(
    vg.Y('cumpercentage:Q',
    title='Cumulative Count',
    axis=vg.Axis(format=".0%")   ),
    text = vg.Text('cumpercentage:Q')
)
# Mark the percentage values on the line with Circle marks
points = base.mark_circle(strokeWidth= 3, color = "#cb4154").encode(
    vg.Y('cumpercentage:Q', axis=None),
    tooltip=['Nome', 'Município', 'Modal', 'Rio', 'OBS', 'Considerar', 'Conteineres/Ano'],
)

# Layer all the elements together 
(bars + line + points).resolve_scale(
    y = 'independent'
).configure_title(
    fontSize=24,
).configure_axis(
    grid=False,
    labelFontSize=10,
    titleFontSize=20,
).add_selection(
    selection
).interactive()

alt.LayerChart(...)

# Considerar

In [9]:
df.loc[df.Nome == 'ALTEROSA', 'Considerar'] = 0

In [10]:
# Topo map
filepath = 'https://raw.githubusercontent.com/tbrugz/geodata-br/master/geojson/geojs-13-mun.json'

topo = vg.Chart(filepath).mark_geoshape(
    stroke="#dd",
    strokeWidth=0.3,
    opacity=0.2,
    color="black",
).project(
    type='mercator'
)

# Demanda
selection = vg.selection_multi(fields=['Rio'], bind='legend')
points = vg.Chart(df[df.Considerar == 1], title="Nós Considerados").mark_line(point=True, size=3).encode(
    vg.Longitude('Longitude:Q',
                ),
    vg.Latitude('Latitude:Q',
               ),
    color= vg.condition(selection,
                        vg.Color('Rio:N', scale=vg.Scale(scheme='category10')),
                        vg.value('lightgray'),
                       ),
    #size=vg.Size('Conteineres/Ano:Q', 
    #             scale=None,
    #             legend=vg.Legend(title='Conteineres por Ano')),
    tooltip=['Sigla', 'Nome', 'Latitude', 'Longitude', 'Município', 'Modal', 'Rio',
             'Latitude_2', 'Longitude_2', 'OBS', 'Considerar', 'Carga_2023',
             'Carga_2024', 'GN', 'GNL', 'Conteineres/Ano']
)

# Mapa + Demanda + Configs
vg.layer(
    topo,
    points,
).properties(
    width=600,
    height=600,
).configure_title(
    fontSize=24,
).configure_axis(
    grid=False,
    labelFontSize=14,
    titleFontSize=20,
).configure_legend(
    titleFontSize=16,
    labelFontSize=14, 
).add_selection(
    selection
)

alt.LayerChart(...)

A fazer:

[DONE] 1) Levantar a matriz de distâncias (Solimões, Madeira e Amazonas) para qualquer origem e destino

2) Calcular tempo de ciclo:

Tempo de ciclo = Tempo de Nav + Tempo de mov de carga

Tempo de Nav = distancia / vel. média

Tempo de Mov de carga = 4 x 5 min

(Demanda por ciclo de 14 dias, 14 dias, 7 dias) - dos nós extremos baseado na velocidade média (ajustado pra cima, múltiplos de 7 são convenientes)

- Tempo de Porto total é de 4 movimentos do conteiner (carreta)

- Estabelecer a frequência de entrega (1, 2, 3 vezes por ciclo)

- Demanda por entrega - aredondar por cima (= Demanda / frequencia)

3) Levantamento da capacidade de navios

4) Geração de rotas (itertools -Python) 
 
 - Limitar trechos entre Portos com alta distância (300 km - verificar qual limite)
 - Capacidade máxima do navio

# Tempo de ciclo

In [11]:
filepath = '../data/distancias.xlsx'
df_dist = pd.read_excel(filepath, index_col=0)

In [12]:
df_dist

,BENJAMIN CONSTANT,TABATINGA,FEIJOAL,BELÉM DO SOLIMÕES,SANTA RITA DO WEILL,SÃO PAULO DE OLIVENÇA,AMATURÁ,SANTO ANTÔNIO DO IÇÁ,TONANTINS,JUTAÍ,...,URUCURITUBA,ITACOATIARA,AUTAZES,NOVA OLINDA DO NORTE,AXINIM,BORBA,NOVO ARIPUANÃ,MANICORÉ,AUXILIADORA,HUMAITÁ
BENJAMIN CONSTANT,0.00,12.25,53.85,98.75,161.80,212.65,299.80,353.40,385.30,524.70,...,1687.85,1650.25,1665.15,1692.05,1729.05,1780.15,1926.15,2073.15,2178.15,2428.15
TABATINGA,12.25,0.00,51.75,96.65,159.70,210.55,297.70,351.30,383.20,522.60,...,1685.75,1648.15,1663.05,1689.95,1726.95,1778.05,1924.05,2071.05,2176.05,2426.05
FEIJOAL,53.85,51.75,0.00,45.00,108.05,158.90,246.05,299.65,331.55,470.95,...,1634.10,1596.50,1611.40,1638.30,1675.30,1726.40,1872.40,2019.40,2124.40,2374.40
BELÉM DO SOLIMÕES,98.75,96.65,45.00,0.00,63.55,114.40,201.55,255.15,287.05,426.45,...,1589.60,1552.00,1566.90,1593.80,1630.80,1681.90,1827.90,1974.90,2079.90,2329.90
SANTA RITA DO WEILL,161.80,159.70,108.05,63.55,0.00,50.90,138.05,191.65,223.55,362.95,...,1526.10,1488.50,1503.40,1530.30,1567.30,1618.40,1764.40,1911.40,2016.40,2266.40
SÃO PAULO DE OLIVENÇA,212.65,210.55,158.90,114.40,50.90,0.00,87.35,140.95,172.85,312.25,...,1475.40,1437.80,1452.70,1479.60,1516.60,1567.70,1713.70,1860.70,1965.70,2215.70
AMATURÁ,299.80,297.70,246.05,201.55,138.05,87.35,0.00,54.10,86.00,225.40,...,1388.55,1350.95,1365.85,1392.75,1429.75,1480.85,1626.85,1773.85,1878.85,2128.85
SANTO ANTÔNIO DO IÇÁ,353.40,351.30,299.65,255.15,191.65,140.95,54.10,0.00,34.10,173.50,...,1336.65,1299.05,1313.95,1340.85,1377.85,1428.95,1574.95,1721.95,1826.95,2076.95
TONANTINS,385.30,383.20,331.55,287.05,223.55,172.85,86.00,34.10,0.00,139.90,...,1303.05,1265.45,1280.35,1307.25,1344.25,1395.35,1541.35,1688.35,1793.35,2043.35
JUTAÍ,524.70,522.60,470.95,426.45,362.95,312.25,225.40,173.50,139.90,0.00,...,1163.20,1125.60,1140.50,1167.40,1204.40,1255.50,1401.50,1548.50,1653.50,1903.50


In [32]:
# Parameters
vel_media = 10 * 1.852 # 10 nós
t_carregamento = 4 * 5 / 60 # 5 minutos para cada movimento (4 movimentos por carga)
f_corredor = 0.9 # porcentagem da capacidade útil 
capac_navio_nominal = 400 # em volume [m3]
capac_carga  = 50 # em volume [m3]

# Rios
rios = ['Amazonas', 'Madeira', 'Solimões']

# Frequencia de visitas
freq_visitas = [3, 3, 2]

In [33]:
# Capacidade do navio
capac_navio = (f_corredor * capac_navio_nominal) / capac_carga

for i in range(0, len(rios)):
    rio = rios[i]
    freq = freq_visitas[i]
    
    # Selecionar localidades "Consideradas" e de dado "Rio"
    loc_rio = df[(df.Considerar == 1) & (df.Rio == rio)]['Nome'].tolist()
    # Maxima dist entre localidades (extremos)
    max_dist = df_dist.loc[loc_rio, loc_rio].max().max()
    # Tempo de Navegação é 2 vezes a distancia dividida pela vel 
    tempo_nav = 2 * max_dist / vel_media
    
    # Demanda total por ano (arredonda para cima)
    demanda_total = m.ceil(df[df.Nome.isin(loc_rio)]['Conteineres/Ano'].sum())
    
    # Viagens redondas por ano (arredonda por cima)
    viagens_redondas = m.ceil(demanda_total / capac_navio)
    
    # Tempo de ciclo
    t_ciclo = (freq * tempo_nav + demanda_total/ (12 * 30 * 24) * t_carregamento)
    
    print(rio)
    print(f'Frequência de entrega: {freq} [-]')
    print(f'Distância máxima: {max_dist} [km]')
    print(f'Tempo de Navegação: {round(tempo_nav, 2)} [h]')
    print(f'Demanda total: {demanda_total} [cargas/ano]')
    print(f'Viagens redondas: {viagens_redondas} [viagens]')
    print(f'Tempo de ciclo: {round(t_ciclo, 2)} [h] ou {round(t_ciclo /24, 2)} [dias] \n')

Amazonas
Frequência de entrega: 3 [-]
Distância máxima: 336.0 [km]
Tempo de Navegação: 36.29 [h]
Demanda total: 4544 [cargas/ano]
Viagens redondas: 632 [viagens]
Tempo de ciclo: 109.03 [h] ou 4.54 [dias] 

Madeira
Frequência de entrega: 3 [-]
Distância máxima: 763.0 [km]
Tempo de Navegação: 82.4 [h]
Demanda total: 2493 [cargas/ano]
Viagens redondas: 347 [viagens]
Tempo de ciclo: 247.29 [h] ou 10.3 [dias] 

Solimões
Frequência de entrega: 2 [-]
Distância máxima: 1449.9 [km]
Tempo de Navegação: 156.58 [h]
Demanda total: 3428 [cargas/ano]
Viagens redondas: 477 [viagens]
Tempo de ciclo: 313.29 [h] ou 13.05 [dias] 

